# Model to classify the report


## Importing Libraries


In [18]:
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multioutput import MultiOutputClassifier

## Loading dataframe


In [19]:
df = pd.read_csv('reports_hp.csv')
df.head()

,title,description,response,data
0,HP bloqueia cartuchos alternativos,"Após atualizar o firmware, a minha HP Pro 9020...",Não Respondida,Há 7 dias
1,Cartuchos HP 63 não funcionam mais após atuali...,Minha impressora HP Pro 9020 parou de aceitar ...,Respondida,Há 5 dias
2,Cartucho HP 63 não reconhecido pela impressora,A assistência da HP não tem dado retorno quant...,Respondida,Há 10 dias
3,HP Pro 9020 trava durante impressão,"Comprei uma impressora HP nos EUA e agora, qua...",Respondida,Há 11 dias
4,Cartucho HP 63 não reconhecido pela impressora,"Comprei uma impressora HP nos EUA e agora, qua...",Respondida,Há 20 dias


## Creating dataframe with labels

In [20]:
labels = {
    'product': ['HP Pro 9020', 'Cartucho HP 63', 'Cartucho HP 63', 'HP Pro 9020'] * 25,
    'department': ['Suporte Técnico', 'Atendimento ao Cliente', 'Atendimento ao Cliente', 'Assistência Técnica'] * 25,
    'sentiment': ['positivo', 'negativo', 'neutro', 'mega-negativo'] * 25
}

labels_df = pd.DataFrame(labels)
labels_df.head()

,product,department,sentiment
0,HP Pro 9020,Suporte Técnico,positivo
1,Cartucho HP 63,Atendimento ao Cliente,negativo
2,Cartucho HP 63,Atendimento ao Cliente,neutro
3,HP Pro 9020,Assistência Técnica,mega-negativo
4,HP Pro 9020,Suporte Técnico,positivo


## Pre-Processing and Split the data


In [21]:
# Text Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['description'])

# Expected variables
y = labels_df[['sentiment', 'product', 'department']] 

# Split the data with train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the length
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (80, 170)
y_train shape: (80, 3)


## Training model


In [22]:

# Initializing the classify with the Logistic Regression
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)


MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000))

## Saving the model trainned


In [23]:
joblib.dump(model, 'model/classify_reports_model.joblib')

print("Model and vectorizer save with suscessfully!")

Model and vectorizer save with suscessfully!


## Model Evaluation

In [24]:
# Sentiment
print("Avaliação de Sentimento:")
print(classification_report(y_test['sentiment'], y_pred[:, 0]))

# Product
print("\nAvaliação de Produto:")
print(classification_report(y_test['product'], y_pred[:, 1]))

# Department
print("\nAvaliação de Departamento:")
print(classification_report(y_test['department'], y_pred[:, 2]))


Avaliação de Sentimento:
               precision    recall  f1-score   support

mega-negativo       0.00      0.00      0.00         3
     negativo       0.00      0.00      0.00         5
       neutro       0.50      0.33      0.40         6
     positivo       0.44      0.67      0.53         6

     accuracy                           0.30        20
    macro avg       0.24      0.25      0.23        20
 weighted avg       0.28      0.30      0.28        20


Avaliação de Produto:
                precision    recall  f1-score   support

Cartucho HP 63       0.55      0.55      0.55        11
   HP Pro 9020       0.44      0.44      0.44         9

      accuracy                           0.50        20
     macro avg       0.49      0.49      0.49        20
  weighted avg       0.50      0.50      0.50        20


Avaliação de Departamento:
                        precision    recall  f1-score   support

   Assistência Técnica       0.00      0.00      0.00         3
Atendimento a

## Testing of the model

In [26]:

test_data = [
    "A minha impressora HP Pro 9020 travou após a atualização do software e não está funcionando.",
    "O cartucho HP 63 não está funcionando corretamente, a impressão fica muito fraca.",
    "Quando imprimo com o cartucho HP 63, as cores não aparecem corretamente na página.",
    "O suporte da HP não está me ajudando com a minha impressora HP Pro 9020 que parou de funcionar."
]

# Vectorizing the test data with TfidfVectorizer
test_data_vectorized = vectorizer.transform(test_data)

# Using the model to predict sentiment, product and departments
predictions = model.predict(test_data_vectorized)

# Show Predictions
for i, text in enumerate(new_data):
    print(f"\nTexto: {text}")
    print(f"Sentimento Predito: {predictions[i, 0]}")
    print(f"Produto Predito: {predictions[i, 1]}")
    print(f"Departamento Predito: {predictions[i, 2]}")



Texto: A minha impressora HP Pro 9020 travou após a atualização do software e não está funcionando.
Sentimento Predito: negativo
Produto Predito: Cartucho HP 63
Departamento Predito: Atendimento ao Cliente

Texto: O cartucho HP 63 não está funcionando corretamente, a impressão fica muito fraca.
Sentimento Predito: mega-negativo
Produto Predito: HP Pro 9020
Departamento Predito: Atendimento ao Cliente

Texto: Quando imprimo com o cartucho HP 63, as cores não aparecem corretamente na página.
Sentimento Predito: mega-negativo
Produto Predito: HP Pro 9020
Departamento Predito: Atendimento ao Cliente

Texto: O suporte da HP não está me ajudando com a minha impressora HP Pro 9020 que parou de funcionar.
Sentimento Predito: neutro
Produto Predito: Cartucho HP 63
Departamento Predito: Atendimento ao Cliente
